**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D, Flatten, Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

`epsilon` is essential during learning for exploration: without `epsilon`, the agent can easily be trapped in a particular suboptimal policy and will apply it repeatedly without trying something else (exploitation). `epsilon` allows to regularly try new policies in order to have a chance to escape from this kind of local minima.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T = 200
temperature = 0.3
epochs_train= 51 # set small when debugging
epochs_test = 11 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array `position` stores the position of the mouse in the grid and the borders: it is full of zeros excepted at the mouse position where the value is 1 and on the borders where it is -1. The mouse can move one cell left, right, up or bottom and "bounces" on walls.

The array `board` stores the island in a grid of the same size: cells with cheese have a value of 0.5, poisonous cells have a value of -1 and empty cells have a value of 0. Values are set to 0 when the mouse passes in a cell.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        # Choose random direction between 0 and 3
        return np.random.randint(0, 4)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0

    for e in range(epochs):
        # Reset the environment
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action using the learned policy
            action = agent.act(state, train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({:.2f})"
              .format(win, lose, score/(1+e)))
    size = env.grid_size - 4
    best_possible_score = size**2 * env.temperature * 0.5 # ignoring that bonuses can be surrounded by maluses
    print(f'Final score: {score/epochs:.2f} (mean best achievable score: {best_possible_score:.2f})')
    print(f'{100*score/epochs/best_possible_score:.2f}%')

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 9.5/19.0. Average score (-9.50)
Win/lose count 9.5/14.0. Average score (-7.00)
Win/lose count 9.5/15.0. Average score (-6.50)
Win/lose count 5.5/15.0. Average score (-7.25)
Win/lose count 8.5/20.0. Average score (-8.10)
Win/lose count 10.5/16.0. Average score (-7.67)
Win/lose count 7.5/11.0. Average score (-7.07)
Win/lose count 5.0/10.0. Average score (-6.81)
Win/lose count 7.0/8.0. Average score (-6.17)
Win/lose count 9.5/17.0. Average score (-6.30)
Win/lose count 7.0/15.0. Average score (-6.45)
Final score: -6.45 (mean best achievable score: 25.35)
-25.46%


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




We have: 
$$
\begin{align*}
Q^\pi(s,a) &= E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
    &= E_{p^{\pi}}[r(s_0,a_0)|s_{0}=s,a_{0}=a] + \gamma E_{p^{\pi}}[\sum_{t=1}^{\infty}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
    &= r(s,a) + \gamma E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a] \\
    &= r(s,a) + \sum_{s', a'} p(s',a'|s,a) \gamma E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a,s_1=s', s_1=a'] \\
    &= r(s,a) + \sum_{s', a'} p(s',a'|s,a) \gamma E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_1=s', s_1=a'] \hbox{ (Markov property)}\\
    &= r(s,a) + \sum_{s', a'} p(s',a'|s,a) \gamma E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_0=s', s_0=a'] \hbox{ (Stationnarity)}\\
    &= r(s,a) + \sum_{s', a'} p(s',a'|s,a) \gamma Q^{\pi}(s',a')\\
    &= r(s,a) + E_{(s',a')\sim p(.|s,a)}[\gamma Q^{\pi}(s',a')]\\
    &= E_{(s',a')\sim p(.|s,a)}[r(s,a) + \gamma Q^{\pi}(s',a')]
\end{align*}
$$

Furthermore:
$$
\begin{align*}
Q^*(s,a) &= \max_{\pi} Q^\pi(s,a) \\
    &= \max_{\pi} E_{(s',a')\sim p(.|s,a)}[r(s,a) + \gamma Q^{\pi}(s',a')] \\
    &= r(s,a) + \gamma \max_{\pi} E_{(s',a')\sim p(.|s,a)}[Q^{\pi}(s',a')] \\
    &= r(s,a) + \gamma \max_{\pi} \sum_{s'} \sum_{a'} p(s'|s,a) \pi(a'|s') Q^{\pi}(s',a') \\
    &\leq r(s,a) + \gamma \sum_{s'}  p(s'|s,a) \max_{\pi} \sum_{a'} \pi(a'|s') Q^{\pi}(s',a') \\
    &\leq r(s,a) + \gamma \sum_{s'}  p(s'|s,a) \max_{\pi} \max_{a'} Q^{\pi}(s',a') \\
    &= r(s,a) + \gamma E_{s'\sim p(.|s,a)}[\max_{a'}Q^{*}(s',a')] \\
    &= E_{s'\sim p(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]
\end{align*}
$$

Moreover, if we define a policy $\bar\pi$ such that for all states $s'$, $\bar\pi(\cdot|s') \in \arg\max_{\pi} \sum_{a'} \pi(a'|s') Q^{\pi}(s',a')$, then we have:
$$
\begin{align*}
\sum_{s'}  p(s'|s,a) \max_{\pi} \sum_{a'} \pi(a'|s') Q^{\pi}(s',a') &= \sum_{s'}  p(s'|s,a) \sum_{a'} \bar\pi(a'|s') Q^{\bar\pi}(s',a')\\
    &\leq \max_{\pi} \sum_{s'} \sum_{a'} p(s'|s,a) \pi(a'|s') Q^{\pi}(s',a')
\end{align*}
$$

Therefore the previous inequalities are equalities and
$$Q^*(s,a) = E_{s'\sim p(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]$$

We finally have:
$$
E_{s'\sim p(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a') - Q^*(s,a)] = 0
$$
So $\mathcal{L}(\theta)$ is a plausible objective to approximate $Q^*$

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()
        self.position = 0
        
    def remember(self, m):
        if len(self.memory) < self.max_memory:
            self.memory.append(m)
        else:
            self.memory[self.position] = m
            self.position  = (self.position + 1) % self.max_memory
            
    def random_access(self):
        return self.memory[np.random.randint(0, len(self.memory))]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over, t=(e%10 == 0 and e > 0))

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.7
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        q_values = self.model.predict(s.reshape(1, *s.shape)).flatten()
        
        return np.argmax(q_values)

    def reinforce(self, s_, n_s_, a_, r_, game_over_, t=False):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        batch = [self.memory.random_access() for i in range(self.batch_size)]
        input_states = np.stack([m[0] for m in batch])
        next_states = np.stack([m[1] for m in batch])
        actions = np.stack([m[2] for m in batch])
        rewards = np.stack([m[3] for m in batch])
        game_overs = np.stack([m[4] for m in batch])
        
        target_q = self.model.predict(input_states)

        target_q[np.arange(self.batch_size), actions] = rewards + (1 - game_overs) * self.discount * np.amax(
                self.model.predict(next_states))

        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        model = Sequential([
            Flatten(input_shape=(5, 5, self.n_state)),
            #Dense(32, activation="relu"),
            Dense(16, activation="relu"),
            Dense(4)
        ])
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=temperature)
agent = DQN_FC(size, lr=.1, epsilon=0.1, memory_size=1024, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train50.mp4'))

Epoch 000/051 | Loss 0.0016 | Win/lose count 4.0/4.0 (0.0)
Epoch 001/051 | Loss 0.0194 | Win/lose count 4.0/2.0 (2.0)
Epoch 002/051 | Loss 0.0100 | Win/lose count 6.0/3.0 (3.0)
Epoch 003/051 | Loss 0.0069 | Win/lose count 7.0/4.0 (3.0)
Epoch 004/051 | Loss 0.0168 | Win/lose count 3.5/1.0 (2.5)
Epoch 005/051 | Loss 0.0069 | Win/lose count 5.0/5.0 (0.0)
Epoch 006/051 | Loss 0.0077 | Win/lose count 4.0/5.0 (-1.0)
Epoch 007/051 | Loss 0.0030 | Win/lose count 1.5/3.0 (-1.5)
Epoch 008/051 | Loss 0.0050 | Win/lose count 6.0/3.0 (3.0)
Epoch 009/051 | Loss 0.0028 | Win/lose count 4.0/3.0 (1.0)
Epoch 010/051 | Loss 0.0090 | Win/lose count 6.5/2.0 (4.5)
Epoch 011/051 | Loss 0.0055 | Win/lose count 6.0/5.0 (1.0)
Epoch 012/051 | Loss 0.0161 | Win/lose count 2.0/1.0 (1.0)
Epoch 013/051 | Loss 0.0024 | Win/lose count 2.0/2.0 (0.0)
Epoch 014/051 | Loss 0.0045 | Win/lose count 4.5/2.0 (2.5)
Epoch 015/051 | Loss 0.0144 | Win/lose count 6.5/3.0 (3.5)
Epoch 016/051 | Loss 0.0124 | Win/lose count 2.0/4.0 (

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [13]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential([
            Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(5, 5, self.n_state)),
            Conv2D(16, kernel_size=(1, 1),
                 activation='relu'),
            Flatten(),
            Dense(4)
        ])
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.), "mse")
        self.model = model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=temperature)
agent = DQN_CNN(size, lr=.1, epsilon=0.1, memory_size=1024, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train50.mp4'))

Epoch 000/051 | Loss 0.0033 | Win/lose count 8.5/5.0 (3.5)
Epoch 001/051 | Loss 0.0124 | Win/lose count 6.5/2.0 (4.5)
Epoch 002/051 | Loss 0.0023 | Win/lose count 4.0/1.0 (3.0)
Epoch 003/051 | Loss 0.0055 | Win/lose count 6.5/3.0 (3.5)
Epoch 004/051 | Loss 0.0018 | Win/lose count 6.5/3.0 (3.5)
Epoch 005/051 | Loss 0.0093 | Win/lose count 9.5/1.0 (8.5)
Epoch 006/051 | Loss 0.0046 | Win/lose count 8.0/3.0 (5.0)
Epoch 007/051 | Loss 0.0032 | Win/lose count 8.5/5.0 (3.5)
Epoch 008/051 | Loss 0.0017 | Win/lose count 7.5/4.0 (3.5)
Epoch 009/051 | Loss 0.0102 | Win/lose count 6.5/1.0 (5.5)
Epoch 010/051 | Loss 0.0030 | Win/lose count 5.5/2.0 (3.5)
Epoch 011/051 | Loss 0.0023 | Win/lose count 14.5/4.0 (10.5)
Epoch 012/051 | Loss 0.0014 | Win/lose count 2.0/1.0 (1.0)
Epoch 013/051 | Loss 0.0080 | Win/lose count 8.5/2.0 (6.5)
Epoch 014/051 | Loss 0.0024 | Win/lose count 4.0/3.0 (1.0)
Epoch 015/051 | Loss 0.0006 | Win/lose count 6.5/4.0 (2.5)
Epoch 016/051 | Loss 0.0021 | Win/lose count 8.5/5.0 (

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [15]:
env = Environment(grid_size=size, max_time=T,temperature=temperature)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2048, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2048, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 3.5/0. Average score (3.50)
Win/lose count 3.5/0. Average score (3.50)
Win/lose count 2.5/0. Average score (3.17)
Win/lose count 3.5/0. Average score (3.25)
Win/lose count 0/0. Average score (2.60)
Win/lose count 3.5/0. Average score (2.75)
Win/lose count 3.0/0. Average score (2.79)
Win/lose count 3.0/0. Average score (2.81)
Win/lose count 0/0. Average score (2.50)
Win/lose count 6.5/0. Average score (2.90)
Win/lose count 0.5/0. Average score (2.68)
Final score: 2.68 (mean best achievable score: 25.35)
10.58%
Test of the FC
Win/lose count 4.0/0. Average score (4.00)
Win/lose count 12.5/0. Average score (8.25)
Win/lose count 4.5/0. Average score (7.00)
Win/lose count 3.0/0. Average score (6.00)
Win/lose count 3.0/0. Average score (5.40)
Win/lose count 3.0/0. Average score (5.00)
Win/lose count 1.0/0. Average score (4.43)
Win/lose count 3.0/0. Average score (4.25)
Win/lose count 6.5/1.0. Average score (4.39)
Win/lose count 7.0/0. Average score (4.65)
Win/lo

In [16]:
HTML(display_videos('cnn_test10.mp4'))

In [17]:
HTML(display_videos('fc_test10.mp4'))

In [18]:
temperature_values = [0.1, 0.2, 0.3, 0.5, 0.8]
for temp in temperature_values:
    print(f"Temperature {temp} (mean best achievable score: {size**2 * temp * 0.5:.2f})")
    env = Environment(grid_size=size, max_time=T, temperature=temp)
    test(agent_cnn,env,epochs_test,prefix=f'cnn_test_temp{temp}_')
    print("\n")
HTML("\n".join(display_videos(f'cnn_test_temp{temp}_0.mp4') for temp in temperature_values))

Temperature 0.1 (mean best achievable score: 8.45)
Win/lose count 3.0/0. Average score (3.00)
Win/lose count 2.0/0. Average score (2.50)
Win/lose count 0/0. Average score (1.67)
Win/lose count 2.0/0. Average score (1.75)
Win/lose count 2.5/0. Average score (1.90)
Win/lose count 2.0/0. Average score (1.92)
Win/lose count 1.0/0. Average score (1.79)
Win/lose count 0.5/0. Average score (1.62)
Win/lose count 0/0. Average score (1.44)
Win/lose count 3.5/0. Average score (1.65)
Win/lose count 2.5/0. Average score (1.73)
Final score: 1.73 (mean best achievable score: 8.45)
20.44%


Temperature 0.2 (mean best achievable score: 16.90)
Win/lose count 6.5/0. Average score (6.50)
Win/lose count 5.5/0. Average score (6.00)
Win/lose count 0/0. Average score (4.00)
Win/lose count 0.5/0. Average score (3.12)
Win/lose count 5.5/0. Average score (3.60)
Win/lose count 0.5/0. Average score (3.08)
Win/lose count 2.0/0. Average score (2.93)
Win/lose count 1.0/0. Average score (2.69)
Win/lose count 2.0/0. Av

The algorithms tend to be blocked when the mouse cannot see any bonus, they then go round in circles and don't explore to find more bonuses. This is all the more a problem when there are few bonuses, ie when the temperature is low, as the probability of being far from a bonus is higher.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [19]:
def train_explore(agent,env,epoch,prefix='', epsilon_decrease=0.9):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        agent.set_epsilon(agent.epsilon * epsilon_decrease)
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{:.1f} ({:.1f})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        super(EnvironmentExploring, self).__init__(grid_size, max_time, temperature)
        
        self.malus_position = np.zeros_like(self.board)
    
    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        
        if train:
            reward -= self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] += 0.1
        self.malus_position[self.x, self.y] = min(0.3, self.malus_position[self.x, self.y])
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                        self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over
    
    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.malus_position = np.zeros_like(self.board)

        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    
## use those samples of code:
#In train explore:
#state, reward, game_over = env.act(action, train=True)

## In Environment exploring:
# You will have to change n_state to 3 because you will use one more layer!
# reward = 0
# if train:
#     reward = -self.malus_position[self.x, self.y]
# self.malus_position[self.x, self.y] = 0.1

# reward = reward + self.board[self.x, self.y]
# # 3 "feature" states instead of 2
# state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
#                                 self.board.reshape(self.grid_size, self.grid_size,1),
#                         self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

In [20]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=temperature)
agent = DQN_CNN(size, lr=.1, epsilon = 0.2, memory_size=2000, batch_size=32, n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore', epsilon_decrease=0.95)
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/051 | Loss 0.0096 | Win/lose count 7.0/48.2 (-41.2)
Epoch 001/051 | Loss 0.0070 | Win/lose count 8.0/49.5 (-41.5)
Epoch 002/051 | Loss 0.0072 | Win/lose count 2.5/52.8 (-50.3)
Epoch 003/051 | Loss 0.0121 | Win/lose count 8.5/44.1 (-35.6)
Epoch 004/051 | Loss 0.0039 | Win/lose count 3.5/53.1 (-49.6)
Epoch 005/051 | Loss 0.0120 | Win/lose count 5.0/50.8 (-45.8)
Epoch 006/051 | Loss 0.0073 | Win/lose count 5.5/50.7 (-45.2)
Epoch 007/051 | Loss 0.0110 | Win/lose count 3.0/50.1 (-47.1)
Epoch 008/051 | Loss 0.0053 | Win/lose count 9.5/39.4 (-29.9)
Epoch 009/051 | Loss 0.0063 | Win/lose count 12.0/40.9 (-28.9)
Epoch 010/051 | Loss 0.0189 | Win/lose count 13.0/37.0 (-24.0)
Epoch 011/051 | Loss 0.0090 | Win/lose count 4.5/50.8 (-46.3)
Epoch 012/051 | Loss 0.0037 | Win/lose count 9.0/45.3 (-36.3)
Epoch 013/051 | Loss 0.0048 | Win/lose count 12.5/38.9 (-26.4)
Epoch 014/051 | Loss 0.0088 | Win/lose count 9.5/40.0 (-30.5)
Epoch 015/051 | Loss 0.0124 | Win/lose count 10.0/42.0 (-32.0)
Epoc

In [21]:
# Evaluation
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=temperature)
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore0.mp4'))

Win/lose count 22.5/2.0. Average score (20.50)
Win/lose count 23.5/0. Average score (22.00)
Win/lose count 17.0/1.0. Average score (20.00)
Win/lose count 24.0/2.0. Average score (20.50)
Win/lose count 19.0/0. Average score (20.20)
Win/lose count 11.5/0. Average score (18.75)
Win/lose count 22.0/1.0. Average score (19.07)
Win/lose count 9.5/0. Average score (17.88)
Win/lose count 22.0/0. Average score (18.33)
Win/lose count 14.5/0. Average score (17.95)
Win/lose count 26.5/1.0. Average score (18.64)
Final score: 18.64 (mean best achievable score: 25.35)
73.52%


In [22]:
temperature_values = [0.1, 0.2, 0.3, 0.5, 0.8]
for temp in temperature_values:
    print(f"Temperature {temp} (mean best achievable score: {size**2 * temp * 0.5:.2f})")
    env = EnvironmentExploring(grid_size=size, max_time=T, temperature=temp)
    test(agent,env,epochs_test,prefix=f'cnn_test_temp{temp}_')
    print("\n")
HTML("\n".join(display_videos(f'cnn_test_temp{temp}_0.mp4') for temp in temperature_values))

Temperature 0.1 (mean best achievable score: 8.45)
Win/lose count 7.0/1.0. Average score (6.00)
Win/lose count 9.0/0. Average score (7.50)
Win/lose count 8.0/0. Average score (7.67)
Win/lose count 6.5/0. Average score (7.38)
Win/lose count 5.5/0. Average score (7.00)
Win/lose count 5.0/1.0. Average score (6.50)
Win/lose count 8.5/0. Average score (6.79)
Win/lose count 9.5/0. Average score (7.12)
Win/lose count 9.0/0. Average score (7.33)
Win/lose count 9.5/0. Average score (7.55)
Win/lose count 9.5/0. Average score (7.73)
Final score: 7.73 (mean best achievable score: 8.45)
91.45%


Temperature 0.2 (mean best achievable score: 16.90)
Win/lose count 15.5/0. Average score (15.50)
Win/lose count 9.0/0. Average score (12.25)
Win/lose count 14.5/1.0. Average score (12.67)
Win/lose count 9.0/0. Average score (11.75)
Win/lose count 9.0/0. Average score (11.20)
Win/lose count 10.0/0. Average score (11.00)
Win/lose count 13.0/0. Average score (11.29)
Win/lose count 9.0/0. Average score (11.00)


The results are much better with exploration, even if the mouse can still stay blocked sometimes when it has already visited all nerby cells. To reduce this phenomenon, I chose to increase the position malus each time the mouse pass through a cell, with a maximum malus of 0.3 (so that it's always better to go through a cell again than to take a malus). Moreover, the decreasing epsilon allows to choose a higher starting epsilon to explore at the beginning while tending to exploitation as epochs go by.

***